# Нормализация, секцияция и чанкинг Markdown

В этом ноутбуке разбиваем Markdown по заголовкам и создаём чанки (~800 токенов, overlap 120). Результат сохраняется в JSONL для последующей векторизации и загрузки в Qdrant.

- Вход: `data/processed/md/{PHB,DMG}.md`
- Выход: `data/processed/chunks/{PHB,DMG}.jsonl`


## Шаг 1. Нормализация Markdown (очистка шума)

Перед чанкингом очищаем Docling/OCR‑артефакты и приводим текст к «готовому к векторизации» виду. Используем объединённый набор правил:

- **Декодирование спецпоследовательностей**: `&amp;` → `&`, `/uniXXXX` → соответствующий символ Юникода.
- **Склейка разнесённых прописных**: `Г ЛАВА`, `Ч АСТЬ`, `D U N G E O N S` → цельные слова.
- **Переносы и разрывы**: `буква-\nбуква` → склейка; внутрисловной `\n` → пробел.
- **Дефис/тире и пробелы**: внутри слов `a - b` → `a-b`; между словами ` - ` → ` — `; лишние пробелы перед знаками препинания убираются.
- **Омографы (латиница↔кириллица)**: латинские `A,B,C,E,H,K,M,O,P,T,X,Y` и строчные аналоги переводятся в кириллицу внутри русских слов.
- **Частный артефакт `fа`**: удаляем начальное латинское `f` перед русской гласной (`fа` → `а`).
- **Сдвоенные слова** после склейки строк: `персонажи персонажи` → `персонажи`.
- **Unicode NFC** и финальная чистка пробелов/пустых строк.

Результат пишем в `data/processed/md_clean/*.md` и используем далее для чанкинга.


In [2]:
# Нормализация исходных Markdown в отдельную директорию
import sys
from pathlib import Path

def _detect_repo_root() -> Path:
    start = Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / 'src').is_dir() and (p / 'data').is_dir():
            return p
    return start

try:
    # Сработает, если код запущен как скрипт, а не ноутбук
    repo_root = Path(__file__).resolve().parent.parent
except NameError:
    # В ноутбуке __file__ нет — ищем корень по структуре проекта
    repo_root = _detect_repo_root()

sys.path.append(str(repo_root / 'src'))

from dnd_rag.core.pipelines import normalize_md_dir_pipeline  # type: ignore

RAW_MD_DIR = repo_root / 'data' / 'processed' / 'md'
CLEAN_MD_DIR = repo_root / 'data' / 'processed' / 'md_clean'

normalized = normalize_md_dir_pipeline(RAW_MD_DIR, CLEAN_MD_DIR)
normalized

[WindowsPath('C:/Users/Shchurov/Хранилище/Документы/05_Программирование/02_projects/personal/dnd_rule_assistant/data/processed/md_clean/DMG.md'),
 WindowsPath('C:/Users/Shchurov/Хранилище/Документы/05_Программирование/02_projects/personal/dnd_rule_assistant/data/processed/md_clean/PHB.md')]

## Шаг 2. Чанкинг очищенного Markdown

Берём файлы из `data/processed/md_clean`, секционируем по заголовкам и режем на перекрывающиеся чанки (~800 токенов, overlap 120). Выход — `data/processed/chunks/*.jsonl` для последующей векторизации/загрузки в Qdrant. Используем `chunk_docs_pipeline` с дефолтной конфигурацией.


In [ ]:
# Импорт и запуск пайплайна чанкинга
import sys
from pathlib import Path

# Добавляем src/ в PYTHONPATH
repo_root = Path(__file__).resolve().parents[1]
sys.path.append(str(repo_root / 'src'))

from dnd_rag.core.pipelines import chunk_docs_pipeline #type: ignore
from dnd_rag.core.config import DEFAULT_CONFIG_PATH #type: ignore

IN_MD_DIR = repo_root / 'data' / 'processed' / 'md_clean'
OUT_CH_DIR = repo_root / 'data' / 'processed' / 'chunks'
CONFIG = DEFAULT_CONFIG_PATH

produced = chunk_docs_pipeline(IN_MD_DIR, OUT_CH_DIR, config_path=CONFIG)
produced


## Мини-поиск (sanity check)

Выполним простой поиск по ключевым словам, чтобы убедиться, что в чанках есть релевантные фрагменты. Для производительного поиска позже будет использован Qdrant + эмбеддинги, здесь — лишь быстрая проверка.


In [ ]:
# Простой keyword-поиск по JSONL
from pathlib import Path
import json

repo_root = Path(__file__).resolve().parents[1]
CH_DIR = repo_root / 'data' / 'processed' / 'chunks'

files = sorted(CH_DIR.glob('*.jsonl'))
rows = []
for fp in files:
    with fp.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))

query = 'что такое преимущество'
terms = [t for t in query.lower().split() if t]

def score(text: str) -> int:
    low = (text or '').lower()
    return sum(low.count(t) for t in terms)

rows.sort(key=lambda r: score(r.get('text', '')), reverse=True)
rows[:5]
